In [2]:
def get_best_epochs(X, y, batch_size):
    
    ## get best epochs
    kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state=1)
    split_idx = kf.split(X, y[:,1])
    
    epochs = []
    for i, idx in enumerate(split_idx):

        train_idx, test_idx = idx

        testY = y[test_idx]
        testX = X[test_idx]

        trainX = X[train_idx]
        trainY = y[train_idx]
        
        clf = AggModel.MultiClassEstimator(gpuid = gpuid, conv1_kernel_size = 13,
                                           batch_size = batch_size,
                                           patience = 20, 
                                           monitor = 'val_loss') #
        
        clf.fit(trainX, trainY, testX, testY)  

        epochs.append(clf._performance.best_epoch)
    
    best_epoch = int(np.mean(epochs))
    print('Best Avg. Epochs: %s' % best_epoch)
    return best_epoch

In [3]:
gpuid = 3

In [4]:
mp = loadmap('/raid/shenwanxiang/bidd-aggmap/paper/transcriptome/ML/RSPC-S.mp')

## 5 fold CV

In [5]:
lst = glob('./tasks_O_z-score/*.h5')


n_fold = 5


res = []
for p in lst:
    
    print('#'*50 + ' %s ' % p + '#'*50 )

    dfx = pd.read_hdf(p, key = 'expression')
    dfy = pd.read_hdf(p, key = 'labels')

    print(p, dfy.value_counts().to_dict())
    
    X = mp.batch_transform(dfx.values)
    Y = pd.get_dummies(dfy).values

    outer = KFold(n_splits = n_fold, shuffle = True, random_state = 42)
    outer_idx = outer.split(X, Y[:,1])

    fold_all = []
    for i, idx in enumerate(outer_idx):

        fold_num = "fold_%s" % str(i).zfill(2) 

        train_idx, test_idx = idx

        testY = Y[test_idx]
        testX = X[test_idx]

        trainX = X[train_idx]
        trainY = Y[train_idx]

        ## training
        print("\nInput train and test X shape is %s, %s  \n" % (trainX.shape,  testX.shape))
        print("Getting the best number of epochs \n")
        
        #optimizied the first fold HPs only

        batch_size = 32
        if len(trainX) < 128:
            batch_size = 16
        best_epochs = get_best_epochs(trainX, trainY, batch_size = batch_size)

        clf = AggModel.MultiClassEstimator(epochs = best_epochs,  gpuid = gpuid, 
                                           conv1_kernel_size = 13,
                                           batch_size = batch_size, verbose=0) #
        clf.fit(trainX, trainY)  #
        
        ## evaluate
        y_true = testY[:,1] 
        y_score = clf.predict_proba(te6stX)[:,1]

        dfr = pd.DataFrame([y_true, y_score]).T
        dfr.columns = ['y_true', 'y_score']
        dfr.index = dfy.iloc[test_idx].index
        acc, auc = score(dfr)
        
        fold_all.append(auc)
        
        res.append([auc, i, p, best_epochs,  batch_size])
        
    print('The auc score for %s is %s.' % (p, np.mean(fold_all)))

  0%|          | 0/554 [00:00<?, ?it/s]

################################################## ./tasks_O_z-score/O_z-score_train_UCEC_grade.h5 ##################################################
./tasks_O_z-score/O_z-score_train_UCEC_grade.h5 {1: 324, 0: 230}


100%|##########| 554/554 [03:02<00:00,  3.03it/s]



Input train and test X shape is (443, 135, 134, 5), (111, 135, 134, 5)  

Getting the best number of epochs 

MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00122: early stopping
MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00125: early stopping
MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00128: early stopping
MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00074: early stopping
MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00092: early stopping
Best Avg. Epochs: 87
MultiClassEstimator(batch_size=32

  0%|          | 0/374 [00:00<?, ?it/s]

...


Input train and test X shape is (425, 135, 134, 5), (107, 135, 134, 5)  

Getting the best number of epochs 

MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00045: early stopping
MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00087: early stopping
MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00102: early stopping
MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00128: early stopping
MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00112: early stopping
Best Avg. Epochs: 73
MultiClassEstimator(batch_size=32

  0%|          | 0/542 [00:00<?, ?it/s]

acc: 0.717, roc-auc: 0.801
The auc score for ./tasks_O_z-score/O_z-score_train_LGG_grade.h5 is 0.7735230468623853.
################################################## ./tasks_O_z-score/O_z-score_train_LUAD_stage.h5 ##################################################
./tasks_O_z-score/O_z-score_train_LUAD_stage.h5 {0: 306, 1: 236}


100%|##########| 542/542 [02:00<00:00,  4.50it/s]



Input train and test X shape is (433, 135, 134, 5), (109, 135, 134, 5)  

Getting the best number of epochs 

MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00055: early stopping
MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00086: early stopping
MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00055: early stopping
MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00068: early stopping
MultiClassEstimator(batch_size=32, conv1_kernel_size=13, gpuid='3', patience=20)

Restoring model weights from the end of the best epoch.

Epoch 00066: early stopping
Best Avg. Epochs: 45
MultiClassEstimator(batch_size=32

In [7]:
dfres = pd.DataFrame(res, columns = ['score', 'fold', 'task', 'best_epoch',  'batch_size'])
dfres.to_csv('./5FCV_results.csv')

In [8]:
dfres.groupby('task').mean().sort_values('score',ascending=False)

,score,fold,best_epoch,batch_size
task,,,,
./tasks_O_z-score/O_z-score_train_UCEC_grade.h5,0.896566,2.0,88.6,32.0
./tasks_O_z-score/O_z-score_train_LGG_grade.h5,0.773523,2.0,66.6,32.0
./tasks_O_z-score/O_z-score_test_HNSC_grade.h5,0.759119,2.0,86.8,32.0
./tasks_O_z-score/O_z-score_train_KIRC_stage.h5,0.756613,2.0,70.2,32.0
./tasks_O_z-score/O_z-score_train_STAD_grade.h5,0.742929,2.0,77.6,32.0
./tasks_O_z-score/O_z-score_train_COAD_stage.h5,0.728824,2.0,105.0,32.0
./tasks_O_z-score/O_z-score_train_UCEC_stage.h5,0.693325,2.0,58.2,32.0
./tasks_O_z-score/O_z-score_train_LIHC_grade.h5,0.691735,2.0,62.8,32.0
./tasks_O_z-score/O_z-score_train_THCA_stage.h5,0.689466,2.0,81.8,32.0


In [9]:
dfres.groupby('task').mean().mean()

score          0.695545
fold           2.000000
best_epoch    72.644444
batch_size    32.000000
dtype: float64

In [10]:
dfres.groupby('task').mean().std()

score          0.071868
fold           0.000000
best_epoch    22.586780
batch_size     0.000000
dtype: float64

In [ ]:
pd.DataFrame(each_fold_results).mean()

In [ ]:
	score	num_samples
task		
UCEC grade	0.895007	554
LGG grade	0.791667	532
KIRC stage	0.775042	544
STAD grade	0.761870	416
HNSC grade	0.725638	504
COAD stage	0.723243	505
THCA stage	0.719254	513
UCEC stage	0.712087	554
CESC grade	0.698648	306
LIHC grade	0.684547	374
SKCM stage	0.683266	249
PAAD grade	0.680912	179
LUSC stage	0.661850	504
LUAD stage	0.653131	542
BRCA stage	0.649536	1134
LIHC stage	0.649238	374
STAD stage	0.616783	416
KIRC grade	0.593755	544